In [2]:
# Install langchain dependencies
!pip install -qU langchain-core
!pip install -qU azure-ai-inference

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00


In [3]:
# Restart kernel after installs so that your environment can access the new
# packages.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [18]:
from typing import Any, Dict, Iterator, List, Mapping, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential

from google.colab import userdata

import os


os.environ["GITHUB_TOKEN"] = userdata.get("GITHUB_TOKEN")


class AzureLC(LLM):

  client: ChatCompletionsClient = None
  model: str = None
  endpoint: str = "https://models.inference.ai.azure.com"
  max_tokens: int = 4096
  temperature: float = 0.7
  top_p: float = 1

  def __init__(self, model_name: str, max_tokens: int, **kwargs: Any):
    super(AzureLC, self).__init__()
    azure_creds = AzureKeyCredential(os.environ["GITHUB_TOKEN"])
    self.client = ChatCompletionsClient(
      endpoint=self.endpoint,
      credential=azure_creds,
      temperature=self.temperature,
      max_tokens=self.max_tokens,
      top_p=self.top_p,
    )
    self.model = model_name
    self.max_tokens = max_tokens

  @property
  def _identifying_params(self) -> Mapping[str, Any]:
    """Get the identifying parameters."""
    return {
      "model": self.model,
      "temperature": self.temperature,
      "top_p": self.top_p,
      "max_tokens": self.max_tokens,
    }

  @property
  def _llm_type(self) -> str:
    """Return type of llm."""
    return "AzureLC"

  def _call(
      self,
      prompt: str,
      stop: Optional[List[str]] = None,
      run_manager: Optional[CallbackManagerForLLMRun] = None,
      **kwargs: Any,
  ) -> str:
    """Call out to Azure models."""
    response = self.client.complete(
      messages=[
        UserMessage(content=prompt),
      ],
      stop=stop,
      model=self.model,
      **kwargs,
    )

    if not response.choices:
      return None

    return response.choices[0].message.content

  def _stream(
      self,
      prompt: str,
      stop: Optional[List[str]] = None,
      run_manager: Optional[CallbackManagerForLLMRun] = None,
      **kwargs: Any,
  ) -> Iterator[GenerationChunk]:
    """Stream from a language model."""
    response = self.client.complete(
      messages=[
        UserMessage(content=prompt),
      ],
      stop=stop,
      model=self.model,
      stream=True,
      **kwargs,
    )

    for update in response:
      if not update.choices:
        break

      chunk = GenerationChunk(text=update.choices[0].delta.content)
      if run_manager:
        run_manager.on_llm_new_token(chunk.text, chunk=chunk)

      yield chunk


In [21]:
llm = AzureLC(model_name="gpt-4o", max_tokens=4096)
print(llm)

AzureLC
Params: {'model': 'gpt-4o', 'temperature': 0.7, 'top_p': 1, 'max_tokens': 4096}


In [23]:
output = llm.invoke("This is a foobar thing")
print(output)

It seems like you might be referring to something specific, but the context is unclear. "Foobar" is often used as a placeholder name in programming and computer science, similar to "foo" and "bar." It can be used to represent variables, functions, or other elements when the actual names are either unknown or irrelevant to the discussion.

If you have a specific question or need help with a particular topic related to "foobar," please provide more details so I can assist you better!
